I created this notebook as part of the Capstone project which I have to obtain my certification "IBM Data science" from Coursera.

In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


## Let's scrap !
<p>First, we are importing the libraries necessary to assign the link and read the source code</p>
<p>After reading the source code, we'll find the class and the balises</p>

In [29]:
# import library
import requests
from bs4 import BeautifulSoup

# assign the link
url ='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
website_url = requests.get(url).text

# read the source code
soup = BeautifulSoup(website_url,'lxml')
print(soup.prettify()[9270:10000])

      <table class="wikitable sortable">
       <tbody>
        <tr>
         <th>
          Postcode
         </th>
         <th>
          Borough
         </th>
         <th>
          Neighbourhood
         </th>
        </tr>
        <tr>
         <td>
          M1A
         </td>
         <td>
          Not assigned
         </td>
         <td>
          Not assigned
         </td>
        </tr>
        <tr>
         <td>
          M2A
         </td>
         <td>
          Not assigned
         </td>
         <td>
          Not assigned
         </td>
        </tr>
        <tr>
         <td>
          M3A
         </td>
         <td>
          <a href="/wiki/North_York" title="North York">
           North York
  


In [30]:
# find class 'wikitable sortable' in the HTML script
my_table = soup.find('table',{'class' : 'wikitable sortable'})
my_table

bs4.element.Tag

In [6]:
# find 'th' to know columns name
col_links = my_table.find_all('th')
col_links

[<th>Postcode</th>, <th>Borough</th>, <th>Neighbourhood
 </th>]

In [7]:
# find 'td' to recover data
data_links = my_table.find_all('td')

# Check
data_links[:9]

[<td>M1A</td>, <td>Not assigned</td>, <td>Not assigned
 </td>, <td>M2A</td>, <td>Not assigned</td>, <td>Not assigned
 </td>, <td>M3A</td>, <td><a href="/wiki/North_York" title="North York">North York</a></td>, <td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
 </td>]

## Let's tranforsm !
<p> We are going to convert the previous data to list, array and finally dataframe. </p>
<p> First, we are going to create a list to have the columns names. </p>
<p> Then, we 'll create an array to organise data. </p>
<p> Finally, we'll concatenate list and array to a dataframe. </p>

In [8]:
# transform to a list
col = []
for link in col_links:
    col.append(link.get_text())
print(col)

['Postcode', 'Borough', 'Neighbourhood\n']


In [9]:
# tranform to a list
data = []

for link in data_links:
    l = link.get_text()
    data.append(l)

# transform to an array
n_col = 3
n_row = int(len(data) / n_col)
data_array = np.array(data).reshape((n_row, n_col))

In [10]:
# finally create a df from columns name and data
data_df = pd.DataFrame(data_array, columns=col)
data_df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n


In [11]:
# clean columns name
data_df.columns = [data_df.columns[0], data_df.columns[1], data_df.columns[2].replace('\n', '')]
data_df.columns

Index(['Postcode', 'Borough', 'Neighbourhood'], dtype='object')

In [12]:
# clean data
data_df['Neighbourhood'] = data_df['Neighbourhood'].str.replace('\n', '')
data_df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


## Let's clean !
<p> We are going to clean the dataframe </p>
<p> For that, we 'll do :
    <ul>
      <li>Copy the original dataframe</li>
      <li>Filter</li>
      <li>Summarize neighbourhood data</li>
    </ul> </p>
<p> What a program ! </p>

In [13]:
# copy of original df
data_df_clean = data_df.copy()

# Ignore cells with a borough that is Not assigned
data_df_clean = data_df_clean[data_df_clean['Borough'] != 'Not assigned']

In [14]:
# groupby to move Neighbourhood values from row to column
data_df_clean = data_df_clean.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(lambda x: pd.Series(list(x))).unstack().reset_index()

# Rename columns name
data_df_clean.columns = ['Postcode'
                         , 'Borough'
                         , 'Neighbourhood1'
                         , 'Neighbourhood2'
                         , 'Neighbourhood3'
                         , 'Neighbourhood4'
                         , 'Neighbourhood5'
                         , 'Neighbourhood6'
                         , 'Neighbourhood7'
                         , 'Neighbourhood8']

# check the transformation
data_df_clean.head()

,Postcode,Borough,Neighbourhood1,Neighbourhood2,Neighbourhood3,Neighbourhood4,Neighbourhood5,Neighbourhood6,Neighbourhood7,Neighbourhood8
0,M1B,Scarborough,Rouge,Malvern,NaN,NaN,NaN,NaN,NaN,NaN
1,M1C,Scarborough,Highland Creek,Rouge Hill,Port Union,NaN,NaN,NaN,NaN,NaN
2,M1E,Scarborough,Guildwood,Morningside,West Hill,NaN,NaN,NaN,NaN,NaN
3,M1G,Scarborough,Woburn,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,M1H,Scarborough,Cedarbrae,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
# create an empty column
data_df_clean['Neighbourhood'] = ''

# Create Neighbourhood list
nbh_list = ['Neighbourhood1'
            , 'Neighbourhood2'
            , 'Neighbourhood3'
            , 'Neighbourhood4'
            , 'Neighbourhood5'
            , 'Neighbourhood6'
            , 'Neighbourhood7'
            , 'Neighbourhood8']

# concatenate Neighbourhood values to new column
for i, n in enumerate(nbh_list):
    if i == 0:
        data_df_clean['Neighbourhood'] = data_df_clean[n].fillna('')
    else:
        data_df_clean['Neighbourhood'] = data_df_clean['Neighbourhood'] + ',' + data_df_clean[n].fillna('')       


# Drop useless columns
data_df_clean.drop(nbh_list, axis=1, inplace=True)

# Replace useless characters with regex
data_df_clean.replace(to_replace=r',{2,}', value='', regex=True, inplace=True)

# Check modifications
data_df_clean.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [16]:
# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough
data_df_clean['Neighbourhood'] = np.where(data_df_clean['Neighbourhood'] == 'Not assigned'
                                          , data_df_clean['Borough']
                                          , data_df_clean['Neighbourhood'])

# Check the transformation with an example
data_df_clean[data_df_clean['Postcode'] == 'M7A']

,Postcode,Borough,Neighbourhood
85,M7A,Queen's Park,Queen's Park


In [17]:
# use the .shape method to print the number of rows
print("The dataframe has", data_df_clean.shape[0], 'rows.')

The dataframe has 103 rows.
